<a href="https://colab.research.google.com/github/550tealeaves/DATA-70500-working-with-data/blob/main/Orig_MergingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging Data Sources

In this notebook, I've demonstrated one way to merge data sources using pandas DataFrames. In this case, I have three global social indicators datasets, all of which contain the same country names, which I can use as an index to zip (or glue) the files together. I'm using the **concat()** function with the **axis=1** flag in order to add variables rather than cases.

I use the **na_values[np.nan]** flag to deal with missing values in the Excel files. When I create the analytical model, I use the **missing='drop'** flag to exclude those cases from the analysis.

In [1]:
# Code block 1a: Libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
# Code block 2a: Reading an Excel spreadsheet to combine
# Country = index
# Also tell what they should do w/ n/a
GlobalIndicators1 = pd.read_excel('http://data.shortell.nyc/files/HumanDevelopment.xlsx', index_col='Country', na_values=[np.nan])
GlobalIndicators1.head()

,HDI Rank,Human Development Index (HDI),Life Expectancy at Birth,Expected Years of Education,Mean Years of Education,Gross National Income (GNI) per Capita,GNI per Capita Rank Minus HDI Rank
Country,,,,,,,
Norway,1,0.944,81.6,17.5,12.6,64992,5
Australia,2,0.935,82.4,20.2,13.0,42261,17
Switzerland,3,0.930,83.0,15.8,12.8,56431,6
Denmark,4,0.923,80.2,18.7,12.7,44025,11
Netherlands,5,0.922,81.6,17.9,11.9,45435,9


In [ ]:
# Code block 2b: Reading another Excel spreadsheet
GlobalIndicators2 = pd.read_excel('http://data.shortell.nyc/files/GenderDevelopment.xlsx', index_col='Country', na_values=[np.nan])
GlobalIndicators2.head()

,GDI Rank,Gender Development Index (GDI),Human Development Index (Female),Human Development Index (Male),Life Expectancy at Birth (Female),Life Expectancy at Birth (Male),Expected Years of Education (Female),Expected Years of Education (Male),Mean Years of Education (Female),Mean Years of Education (Male),Estimated Gross National Income per Capita (Female),Estimated Gross National Income per Capita (Male)
Country,,,,,,,,,,,,
Norway,1,0.996,0.940,0.944,83.6,79.5,18.2,16.8,12.7,12.5,57140.0,72825.0
Australia,2,0.976,0.922,0.945,84.5,80.3,20.7,19.7,13.1,12.9,33688.0,50914.0
Switzerland,3,0.950,0.898,0.945,85.0,80.8,15.7,15.9,11.5,13.1,44132.0,69077.0
Denmark,4,0.977,0.912,0.934,82.2,78.3,19.3,18.1,12.8,12.7,36439.0,51727.0
Netherlands,5,0.947,0.893,0.943,83.3,79.7,18.0,17.9,11.6,12.2,29500.0,61641.0


In [ ]:
# Code block 2c: Reading a third Excel spreadsheet
GlobalIndicators3 = pd.read_excel('http://data.shortell.nyc/files/GenderInequality.xlsx', index_col='Country', na_values=[np.nan])
GlobalIndicators3.head()

,GII Rank,Gender Inequality Index (GII),Maternal Mortality Ratio,Adolescent Birth Rate,Percent Representation in Parliament,Population with Secondary Education (Female),Population with Secondary Education (Male),Labour Force Participation Rate (Female),Labour Force Participation Rate (Male)
Country,,,,,,,,,
Norway,1,0.067,4.0,7.8,39.6,97.4,96.7,61.2,68.7
Australia,2,0.110,6.0,12.1,30.5,94.3,94.6,58.8,71.8
Switzerland,3,0.028,6.0,1.9,28.5,95.0,96.6,61.8,74.9
Denmark,4,0.048,5.0,5.1,38.0,95.5,96.6,58.7,66.4
Netherlands,5,0.062,6.0,6.2,36.9,87.7,90.5,58.5,70.6


In [ ]:
# Code block 2d: Reading a fourth spreadsheet
# This dataset has similar structure as the first = logic to combining them
# Used country as index b/c it's listed in both dataset
GlobalIndicators4 = pd.read_csv('http://data.shortell.nyc/files/world-happiness-report-2019.csv', index_col='Country (region)')
GlobalIndicators4.head()

,Ladder,SD of Ladder,Positive affect,Negative affect,Social support,Freedom,Corruption,Generosity,Log of GDP\nper capita,Healthy life\nexpectancy
Country (region),,,,,,,,,,
Finland,1,4,41.0,10.0,2.0,5.0,4.0,47.0,22.0,27.0
Denmark,2,13,24.0,26.0,4.0,6.0,3.0,22.0,14.0,23.0
Norway,3,8,16.0,29.0,3.0,3.0,8.0,11.0,7.0,12.0
Iceland,4,9,3.0,3.0,1.0,7.0,45.0,3.0,15.0,13.0
Netherlands,5,1,12.0,25.0,15.0,19.0,12.0,7.0,12.0,18.0


Here, we will combine the three new DataFrames we've just created into a new DataFrame by concatinating the parts. The flag, **axis=1**, allows us to stack the three files side-by-side, as it were, rather than on top of each other. We can do this because all of them are indexed by Country. We can now use variables from any of the data sources in our analysis.

In [ ]:
# Code block 3: Merging the DataFrames using the concat method in pandas
# indexed all files by country and uses concat method to combine them
GlobalIndicatorsTotal = pd.concat([GlobalIndicators1, GlobalIndicators2, GlobalIndicators3, GlobalIndicators4], axis=1)
GlobalIndicatorsTotal.head() # shows new measures added

,HDI Rank,Human Development Index (HDI),Life Expectancy at Birth,Expected Years of Education,Mean Years of Education,Gross National Income (GNI) per Capita,GNI per Capita Rank Minus HDI Rank,GDI Rank,Gender Development Index (GDI),Human Development Index (Female),Human Development Index (Male),Life Expectancy at Birth (Female),Life Expectancy at Birth (Male),Expected Years of Education (Female),Expected Years of Education (Male),Mean Years of Education (Female),Mean Years of Education (Male),Estimated Gross National Income per Capita (Female),Estimated Gross National Income per Capita (Male),GII Rank,Gender Inequality Index (GII),Maternal Mortality Ratio,Adolescent Birth Rate,Percent Representation in Parliament,Population with Secondary Education (Female),Population with Secondary Education (Male),Labour Force Participation Rate (Female),Labour Force Participation Rate (Male),Ladder,SD of Ladder,Positive affect,Negative affect,Social support,Freedom,Corruption,Generosity,Log of GDP\nper capita,Healthy life\nexpectancy
Norway,1.0,0.944,81.6,17.5,12.6,64992.0,5.0,1.0,0.996,0.940,0.944,83.6,79.5,18.2,16.8,12.7,12.5,57140.0,72825.0,1.0,0.067,4.0,7.8,39.6,97.4,96.7,61.2,68.7,3.0,8.0,16.0,29.0,3.0,3.0,8.0,11.0,7.0,12.0
Australia,2.0,0.935,82.4,20.2,13.0,42261.0,17.0,2.0,0.976,0.922,0.945,84.5,80.3,20.7,19.7,13.1,12.9,33688.0,50914.0,2.0,0.110,6.0,12.1,30.5,94.3,94.6,58.8,71.8,11.0,26.0,47.0,37.0,7.0,17.0,13.0,6.0,18.0,10.0
Switzerland,3.0,0.930,83.0,15.8,12.8,56431.0,6.0,3.0,0.950,0.898,0.945,85.0,80.8,15.7,15.9,11.5,13.1,44132.0,69077.0,3.0,0.028,6.0,1.9,28.5,95.0,96.6,61.8,74.9,6.0,11.0,44.0,21.0,13.0,11.0,7.0,16.0,8.0,4.0
Denmark,4.0,0.923,80.2,18.7,12.7,44025.0,11.0,4.0,0.977,0.912,0.934,82.2,78.3,19.3,18.1,12.8,12.7,36439.0,51727.0,4.0,0.048,5.0,5.1,38.0,95.5,96.6,58.7,66.4,2.0,13.0,24.0,26.0,4.0,6.0,3.0,22.0,14.0,23.0
Netherlands,5.0,0.922,81.6,17.9,11.9,45435.0,9.0,5.0,0.947,0.893,0.943,83.3,79.7,18.0,17.9,11.6,12.2,29500.0,61641.0,5.0,0.062,6.0,6.2,36.9,87.7,90.5,58.5,70.6,5.0,1.0,12.0,25.0,15.0,19.0,12.0,7.0,12.0,18.0


In our model, we are trying to explain variation in aggregate quality of life. We're going to use five predictors, all measuring social aspects of gender inequality. That is, social policy contributes to why some nations are higher on these dimensions than others. In the linear model, we are using each of these predictors as control variables for the others. In this way, we get an estimate of the unique effect of each predictor.

This is important, because these proxies of social policy intersect. That is, nations that have policy to encourage and support education for girls and women tend to have policies that give women more control over their reproductive rights, encourage labor force participation, and have space in the political sphere for women to run for elected office. But in the model, we can estimate the unique effects--the variation that a particular predictor can account for that the others cannot.

In [ ]:
# Code block 4: Using the merged data in a linear model
Y = GlobalIndicatorsTotal['Ladder'] # A measure of overall quality of life
X = GlobalIndicatorsTotal[['Mean Years of Education (Female)', 'Labour Force Participation Rate (Female)', 'Adolescent Birth Rate', 'Percent Representation in Parliament']]
X = sm.add_constant(X)
model0 = sm.OLS(Y, X, missing='drop').fit()
print(model0.summary())

                            OLS Regression Results                            
Dep. Variable:                 Ladder   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     37.74
Date:                Tue, 28 Sep 2021   Prob (F-statistic):           7.94e-21
Time:                        16:30:04   Log-Likelihood:                -646.93
No. Observations:                 133   AIC:                             1304.
Df Residuals:                     128   BIC:                             1318.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

Let's look at the results of our model. First, we can see that the overall significance test is above the threshold. In the first part of the table, you can see the F-statistic and the probability associated with it [Prob (F-statistic)]. The probability for this test is less than the conventional criterion, 0.05, so we can proceed with the interpretation of the model.

The goodness of fit test, R-squared, tells us the proportion of variation in aggregate happiness (our DV) that we can explain with the model. In this case, 68% is a good fit. The five predictors do a good job explaining variation among countries on our DV.

Next, we look at the significance tests for the individual predictors. In this case, the probabilities (show in the P>|t| column in the second part of the table) are  less than the conventional criterion, 0.05, on some independent variables but not others. We can interpret the unique effects only for the predictors that are reliable, as indicated by the significance test.

The coefficients (coef) are in the units of the dependent variable. In this case, with our DV, that would be ranks. So each coefficient tells us the change in aggregate happiness rank (which we can call "steps" or ranks) we would predict with a one unit increase in that independent variable.

The predictors that don't have a significant unique effect are still important in the model, since they serve as control variables for the significan unique effects.

We can use the results to formulate a sociological argument about the relationship between gender equality in nations and aggregate happiness.

In [ ]:
X.corr()

,const,Mean Years of Education (Female),Labour Force Participation Rate (Female),Adolescent Birth Rate,Percent Representation in Parliament,Life Expectancy at Birth (Female)
const,NaN,NaN,NaN,NaN,NaN,NaN
Mean Years of Education (Female),NaN,1.000000,-0.115538,-0.672112,0.152759,0.755096
Labour Force Participation Rate (Female),NaN,-0.115538,1.000000,0.324286,0.180077,-0.279472
Adolescent Birth Rate,NaN,-0.672112,0.324286,1.000000,-0.040051,-0.737655
Percent Representation in Parliament,NaN,0.152759,0.180077,-0.040051,1.000000,0.165614
Life Expectancy at Birth (Female),NaN,0.755096,-0.279472,-0.737655,0.165614,1.000000


In [ ]:
# Code block 4b: Using the merged data in a linear model
Y = GlobalIndicatorsTotal['Ladder'] # A measure of overall quality of life
X = GlobalIndicatorsTotal['HDI Rank']
X = sm.add_constant(X)
model2 = sm.OLS(Y, X, missing='drop').fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 Ladder   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     307.6
Date:                Tue, 28 Sep 2021   Prob (F-statistic):           2.12e-36
Time:                        16:23:43   Log-Likelihood:                -628.91
No. Observations:                 135   AIC:                             1262.
Df Residuals:                     133   BIC:                             1268.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         15.0138      4.151      3.617      0.0

## More on Merging Data Sources

In this example, I'm combining a number of data files from the NYC Department of Education. They are all using the same aggregate units: schools. The DoE has a standardized identifier for schools in the system. It isn't always given the same name, but if we identify it in the data source, we can use it as the basis for zipping the files together. These files come from the NYC OpenData archive, https://opendata.cityofnewyork.us/ . In this instance, they are in JSON format.

In [ ]:
# Code block 5a: Reading a json file
DOEgraduation = pd.read_json('https://data.cityofnewyork.us/resource/nb39-jx2v.json')
DOEgraduation.head()

,dbn,school_name,demographic_category,demographic_variable,cohort_year,cohort,total_cohort,total_grads,total_grads_of_cohort,total_regents,total_regents_of_cohort,total_regents_of_grads,advanced_regents,advanced_regents_of_cohort,advanced_regents_of_grads,regents_w_o_advanced,regents_w_o_advanced_of_cohort,regents_w_o_advanced_of_grads,local,local_of_cohort,local_of_grads,still_enrolled,still_enrolled_of_cohort,dropped_out,dropped_out_of_cohort
0,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4
1,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2012,4 year August,44,24.0,54.5,20.0,45.5,83.3,1.0,2.3,4.2,19.0,43.2,79.2,4.0,9.1,16.7,10.0,22.7,10.0,22.7
2,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2011,4 year August,73,46.0,63.0,41.0,56.2,89.1,0.0,0.0,0.0,41.0,56.2,89.1,5.0,6.8,10.9,18.0,24.7,7.0,9.6
3,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2010,4 year August,61,26.0,42.6,26.0,42.6,100.0,1.0,1.6,3.8,25.0,41.0,96.2,0.0,0.0,0.0,18.0,29.5,17.0,27.9
4,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2009,4 year August,85,49.0,57.6,44.0,51.8,89.8,0.0,0.0,0.0,44.0,51.8,89.8,5.0,5.9,10.2,28.0,32.9,8.0,9.4


In [ ]:
# Code block 5b: Reading another json file
DOEregents = pd.read_json('https://data.cityofnewyork.us/resource/cbrh-qrk4.json')
DOEregents.head()

,lookup,school_dbn,schoolname,schooltype,schoollevel,regents_exam,year,total_tested,mean_score,number_scoring_below_65,percent_scoring_below_65,number_scoring_65_or_above,percent_scoring_65_or_above,number_scoring_80_or_above,percent_scoring_80_or_above,number_scoring_cr,percent_scoring_cr
0,01M0342017Common Core Algebra,01M034,P.S. 034 Franklin D. Roosevelt,General Academic,K-8,Common Core Algebra,2017,4,s,s,s,s,s,s,s,na,na
1,01M0342015Living Environment,01M034,P.S. 034 Franklin D. Roosevelt,General Academic,K-8,Living Environment,2015,16,77.9,1,6.3,15,93.8,7,43.8,na,na
2,01M0342016Living Environment,01M034,P.S. 034 Franklin D. Roosevelt,General Academic,K-8,Living Environment,2016,9,74.0,1,11.1,8,88.9,2,22.2,na,na
3,01M1402016Common Core Algebra,01M140,P.S. 140 Nathan Straus,General Academic,K-8,Common Core Algebra,2016,3,s,s,s,s,s,s,s,na,na
4,01M1402017Common Core Algebra,01M140,P.S. 140 Nathan Straus,General Academic,K-8,Common Core Algebra,2017,2,s,s,s,s,s,s,s,na,na


In [ ]:
# Code block 5c: Combining the DoE json files using the merge method in pandas
# have to specify the variable that they will be matched when using the merge method
# Index is always left most column
DOEdata = pd.merge(left=DOEgraduation, right=DOEregents, left_on='dbn', right_on='school_dbn')
DOEdata.head()

,dbn,school_name,demographic_category,demographic_variable,cohort_year,cohort,total_cohort,total_grads,total_grads_of_cohort,total_regents,total_regents_of_cohort,total_regents_of_grads,advanced_regents,advanced_regents_of_cohort,advanced_regents_of_grads,regents_w_o_advanced,regents_w_o_advanced_of_cohort,regents_w_o_advanced_of_grads,local,local_of_cohort,local_of_grads,still_enrolled,still_enrolled_of_cohort,dropped_out,dropped_out_of_cohort,lookup,school_dbn,schoolname,schooltype,schoollevel,regents_exam,year,total_tested,mean_score,number_scoring_below_65,percent_scoring_below_65,number_scoring_65_or_above,percent_scoring_65_or_above,number_scoring_80_or_above,percent_scoring_80_or_above,number_scoring_cr,percent_scoring_cr
0,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2015,5,s,s,s,s,s,s,s,na,na
1,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922016Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2016,5,s,s,s,s,s,s,s,na,na
2,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Common Core Algebra,01M292,Orchard Collegiate Academy,General Academic,High school,Common Core Algebra,2015,12,52.3,11,91.7,1,8.3,0,0.0,0,0.0
3,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922016Common Core Algebra,01M292,Orchard Collegiate Academy,General Academic,High school,Common Core Algebra,2016,50,63.9,25,50.0,25,50.0,3,6.0,15,30.0
4,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922017Common Core Algebra,01M292,Orchard Collegiate Academy,General Academic,High school,Common Core Algebra,2017,53,67.5,15,28.3,38,71.7,6,11.3,29,54.7


In [ ]:
# Code block 5d: Reading another json file
DOEdemographics = pd.read_json('https://data.cityofnewyork.us/resource/s52a-8aq6.json') # https://data.cityofnewyork.us/Education/2013-2018-Demographic-Snapshot-School/s52a-8aq6
DOEdemographics.head()

,dbn,school_name,year,total_enrollment,grade_pk_half_day_full_day,grade_k,grade_1,grade_2,grade_3,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,female_1,female_2,male_1,male_2,asian_1,asian_2,black_1,black_2,hispanic_1,hispanic_2,multiple_race_categories_not_represented_1,multiple_race_categories_not_represented_2,white_1,white_2,students_with_disabilities_1,students_with_disabilities_2,english_language_learners_1,english_language_learners_2,poverty_1,poverty_2,economic_need_index
0,01M015,P.S. 015 Roberto Clemente,2013-14,190,26,39,39,21,16,26,23,0,0,0,0,0,0,0,93,48.9,97,51.1,9,4.7,72,37.9,104,54.7,2,1.1,3,1.6,65,34.2,19,10.0,171,90.0,No Data
1,01M015,P.S. 015 Roberto Clemente,2014-15,183,18,27,47,31,19,17,24,0,0,0,0,0,0,0,84,45.9,99,54.1,8,4.4,65,35.5,107,58.5,1,0.5,2,1.1,64,35.0,17,9.3,169,92.3,93.5%
2,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,18,0,0,0,0,0,0,0,83,47.2,93,52.8,9,5.1,57,32.4,105,59.7,3,1.7,2,1.1,60,34.1,16,9.1,149,84.7,89.6%
3,01M015,P.S. 015 Roberto Clemente,2016-17,178,17,28,33,27,31,24,18,0,0,0,0,0,0,0,83,46.6,95,53.4,14,7.9,51,28.7,105,59.0,4,2.2,4,2.2,51,28.7,12,6.7,152,85.4,89.2%
4,01M015,P.S. 015 Roberto Clemente,2017-18,190,17,28,32,33,23,31,26,0,0,0,0,0,0,0,99,52.1,91,47.9,20,10.5,52,27.4,110,57.9,2,1.1,6,3.2,45,23.7,8,4.2,161,84.7,89.0%


In [ ]:
# Code block 5e: Combining the DataFrames for analysis
DOEdata2 = pd.merge(left=DOEdata, right=DOEdemographics, left_on='dbn', right_on='dbn')
DOEdata2.head()

,dbn,school_name_x,demographic_category,demographic_variable,cohort_year,cohort,total_cohort,total_grads,total_grads_of_cohort,total_regents,total_regents_of_cohort,total_regents_of_grads,advanced_regents,advanced_regents_of_cohort,advanced_regents_of_grads,regents_w_o_advanced,regents_w_o_advanced_of_cohort,regents_w_o_advanced_of_grads,local,local_of_cohort,local_of_grads,still_enrolled,still_enrolled_of_cohort,dropped_out,dropped_out_of_cohort,lookup,school_dbn,schoolname,schooltype,schoollevel,regents_exam,year_x,total_tested,mean_score,number_scoring_below_65,percent_scoring_below_65,number_scoring_65_or_above,percent_scoring_65_or_above,number_scoring_80_or_above,percent_scoring_80_or_above,number_scoring_cr,percent_scoring_cr,school_name_y,year_y,total_enrollment,grade_pk_half_day_full_day,grade_k,grade_1,grade_2,grade_3,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,female_1,female_2,male_1,male_2,asian_1,asian_2,black_1,black_2,hispanic_1,hispanic_2,multiple_race_categories_not_represented_1,multiple_race_categories_not_represented_2,white_1,white_2,students_with_disabilities_1,students_with_disabilities_2,english_language_learners_1,english_language_learners_2,poverty_1,poverty_2,economic_need_index
0,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2015,5,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2013-14,323,0,0,0,0,0,0,0,13,19,33,59,68,68,63,129,39.9,194,60.1,44,13.6,84,26.0,172,53.3,13,4.0,10,3.1,107,33.1,64,19.8,271,83.9,No Data
1,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2015,5,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2014-15,255,0,0,0,0,0,0,0,17,14,19,59,52,36,58,96,37.6,159,62.4,32,12.5,66,25.9,142,55.7,6,2.4,9,3.5,83,32.5,30,11.8,227,89.0,83.9%
2,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2015,5,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2015-16,185,0,0,0,0,0,0,0,0,14,11,42,49,37,32,68,36.8,117,63.2,22,11.9,49,26.5,106,57.3,2,1.1,6,3.2,64,34.6,26,14.1,163,88.1,86.7%
3,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2015,5,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2016-17,140,0,0,0,0,0,0,0,0,0,0,40,35,31,34,53,37.9,87,62.1,21,15.0,34,24.3,77,55.0,1,0.7,7,5.0,38,27.1,20,14.3,128,91.4,84.4%
4,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25.0,69.4,23.0,63.9,92.0,0.0,0.0,0.0,23.0,63.9,92.0,2.0,5.6,8.0,3.0,8.3,7.0,19.4,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,2015,5,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2017-18,171,0,0,0,0,0,0,0,0,0,0,72,40,30,29,81,47.4,90,52.6,19,11.1,43,25.1,97,56.7,4,2.3,8,4.7,42,24.6,23,13.5,156,91.2,89.1%


## Data Cleaning

In [ ]:
# Code block 6a: Data cleaning
# These are the columns (variables) that are typed as objects, which in this context means strings (text). So if any of the variables we expect to
# be numeric show up here, we need to do some data cleaning.
DOEdata2.select_dtypes(object)


,dbn,school_name_x,demographic_category,demographic_variable,cohort,lookup,school_dbn,schoolname,schooltype,schoollevel,regents_exam,mean_score,number_scoring_below_65,percent_scoring_below_65,number_scoring_65_or_above,percent_scoring_65_or_above,number_scoring_80_or_above,percent_scoring_80_or_above,number_scoring_cr,percent_scoring_cr,school_name_y,year_y,economic_need_index
0,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,4 year August,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2013-14,No Data
1,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,4 year August,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2014-15,83.9%
2,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,4 year August,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2015-16,86.7%
3,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,4 year August,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2016-17,84.4%
4,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,4 year August,01M2922015Algebra2/Trigonometry,01M292,Orchard Collegiate Academy,General Academic,High school,Algebra2/Trigonometry,s,s,s,s,s,s,s,na,na,Orchard Collegiate Academy,2017-18,89.1%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139081,02M374,GRAMERCY ARTS HIGH SCHOOL,All Students,All Students,4 year June,02M3742017U.S. History and Government,02M374,Gramercy Arts High School,General Academic,High school,U.S. History and Government,72.9,35,27.8,91,72.2,48,38.1,na,na,Gramercy Arts High School,2013-14,No Data
139082,02M374,GRAMERCY ARTS HIGH SCHOOL,All Students,All Students,4 year June,02M3742017U.S. History and Government,02M374,Gramercy Arts High School,General Academic,High school,U.S. History and Government,72.9,35,27.8,91,72.2,48,38.1,na,na,Gramercy Arts High School,2014-15,66.6%
139083,02M374,GRAMERCY ARTS HIGH SCHOOL,All Students,All Students,4 year June,02M3742017U.S. History and Government,02M374,Gramercy Arts High School,General Academic,High school,U.S. History and Government,72.9,35,27.8,91,72.2,48,38.1,na,na,Gramercy Arts High School,2015-16,64.9%
139084,02M374,GRAMERCY ARTS HIGH SCHOOL,All Students,All Students,4 year June,02M3742017U.S. History and Government,02M374,Gramercy Arts High School,General Academic,High school,U.S. History and Government,72.9,35,27.8,91,72.2,48,38.1,na,na,Gramercy Arts High School,2016-17,64.1%


In [ ]:
#Issue with the data ended up being that some schools used # and "S"
# Solevd it in the frame

# Code block 6b: More data cleaning
DOEdata2.loc[DOEdata2.percent_scoring_80_or_above == 's', ['percent_scoring_80_or_above']] = np.nan # Here, we're turning a string to the standard numeric missing value

DOEdata2['percent_scoring_80_or_above'] = DOEdata2['percent_scoring_80_or_above'].apply(pd.to_numeric, downcast='float', errors='coerce')
# Now we can convert the variable from a string to a numeric using the to_numeric method in pandas and applying the results to the DataFrame

DOEdata2['mean_score'] = DOEdata2['mean_score'].apply(pd.to_numeric, downcast='float', errors='coerce')
# We need to convert any string variables (objects) that we want to use in our linear model - it isnot text



In [ ]:
# Code block 7: Another linear model
Y = DOEdata2['mean_score']
X = DOEdata2[['poverty_2', 'english_language_learners_2', 'total_enrollment']]
X = sm.add_constant(X)
model1 = sm.OLS(Y, X, missing='drop').fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             mean_score   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                 2.442e+04
Date:                Tue, 27 Oct 2020   Prob (F-statistic):               0.00
Time:                        01:02:40   Log-Likelihood:            -4.2547e+05
No. Observations:              118566   AIC:                         8.509e+05
Df Residuals:                  118562   BIC:                         8.510e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

## Exercise

1. Read in these data files from the Department of Education or another data source, using the standardized school ID variable, as shown in the parts of code block 5.

2. Clean the data by converting the variables that should be numeric.

3. Create a linear model to explain some aspect of these education data.

4. Write a one paragraph explanation of the insights from your model.